In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
import astropy.io.fits as fits
from astropy.table import Table,join,vstack,unique,QTable
import sys

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from pyutils import *
from dataloc import *
from bgs_helpers import *
from groupcatalog import *
from nnanalysis import *
import plotting as pp
from footprintmanager import FootprintManager
import catalog_definitions as cat

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

In [ ]:
table = Table.read(IAN_BGS_Y3_MERGED_FILE_LOA_SV3CUT, format='fits')

In [ ]:
# Select ones with 'Z' 
loa_missing = determine_unobserved_from_z(table['Z'])
sv3_not_missing = ~determine_unobserved_from_z(table['Z_SV3'])

table['Z', 'Z_SV3', 'Z_PHOT', 'Z_PHOT_FAKE'][loa_missing & sv3_not_missing]

In [ ]:
# Question: how does improving z_phot affect my lost galaxy handling?
# Idea: make a new z_phot_fake1 column that moves zphot half the distance to z_spec within the SV3 sample.
# Then quick check is to re-run existing stuff as-is and see what happens.
# Longer check is then to re-calibrate the method with new z_phot_fake1 and see what happens.
#table = Table.read(IAN_BGS_Y3_MERGED_FILE_LOA_SV3CUT, format='fits')
#table2 = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')

# First, we need to join with IAN_BGS_SV3_MERGED_FILE on TARGETID to get z_spec for the lost galsaxies.
#sv3_table = Table.read(IAN_BGS_SV3_MERGED_FILE, format='fits')
#sv3_table = sv3_table['TARGETID', 'Z']  # Keep only TARGETID and Z (spec-z)
#table = join(table, sv3_table, keys='TARGETID', table_names=['_LOA', '_SV3'], join_type='inner')

def add_zphot_fake(table):
    # Drop column if it already exists
    if 'Z_PHOT_FAKE' in table.colnames:
        table.remove_column('Z_PHOT_FAKE')
    # Identify galaxies with a spectroscopic redshift
    has_zspec = ~determine_unobserved_from_z(table['Z_SV3'])
    print(f"Number of galaxies with z_spec: {np.sum(has_zspec)}")
    # Calculate the difference between z_phot and z_spec
    delta_z = table['Z_SV3'][has_zspec]- table['Z_PHOT'][has_zspec] # 0.1  0.12 -> -0.02. So fake is 0.11

    # Create the new fake z_phot column
    # Initialize with the original Z_PHOT
    table['Z_PHOT_FAKE'] = table['Z_PHOT']
    # For galaxies with z_spec, move z_phot halfway to z_spec
    table['Z_PHOT_FAKE'][has_zspec] = table['Z_PHOT'][has_zspec] + delta_z / 2.0


add_zphot_fake(table)

# Write out the modified tables
table.write(IAN_BGS_Y3_MERGED_FILE_LOA_SV3CUT, format='fits', overwrite=True)
#table2.write(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits', overwrite=True)

In [ ]:
truth_cat = deserialize(cat.bgs_sv3_pz_2_6_10p_c2)

test_cat_1 = BGSGroupCatalog(
    "Test With Improved z_phot", 
    Mode.PHOTOZ_PLUS_v2, 
    19.5, 
    21.0, 
    num_passes=1, 
    data_cut='Y3-Loa-SV3Cut', 
    sdss_fill=False, 
    extra_params=cat.PZP_PARAMS_VFUTURE, 
    gfprops=cat.GF_PROPS_BGS_COLORS_C2.copy())


# TODO go modify Z_PHOT to Z_PHOT_FAKE in pre_process_BGS(...)
test_cat_1.preprocess()
test_cat_1.run_group_finder(popmock=False)
#test_cat_1.calc_wp_for_mock()
test_cat_1.postprocess()
#test_cat_1.chisqr()



In [ ]:
current_test = deserialize(cat.bgs_y3_like_sv3_pz_2_6_c2)
test_cat_1.marker = '--'
test_cat_1.color = 'r'
current_test.name = "Current Method"
truth_cat.name = "~Truth"

sets = [current_test, test_cat_1]

truth_df = truth_cat.all_data
for s in sets:
    print(s.name)
    s.get_true_z_from(truth_df)
    s.refresh_df_views()

In [ ]:
#pp.luminosity_function_plots(*sets)
pp.plots(truth_cat, *sets)

pp.test_purity_and_completeness(*sets, truth_catalog=truth_cat, lost_only=False)
pp.purity_complete_plots(*sets)

#for s in sets:
#    pp.proj_clustering_plot(s)
#    s.chisqr()

pp.correct_redshifts_assigned_plot(*sets)